In [ ]:
#!pip install tensorflow
#!pip install tqdm
#!pip install --upgrade music21
#!pip install pandas
#!pip install glob
#!pip install KERAS


In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import glob
from tqdm import tqdm
from music21 import *
from tensorflow.python.ops import control_flow_ops
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [ ]:
s = converter.parse('elise.mid')

In [2]:
notes = []
for file in glob.glob("*.mid"):
    midi = converter.parse(file)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

In [3]:
n_vocab=len(set(notes))
print(n_vocab)

162


In [4]:
sequence_length = 150
# get all pitch names
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / float(n_vocab)
network_output = np_utils.to_categorical(network_output)

In [5]:
model = Sequential()
model.add(LSTM(256,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [6]:
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger-2MIDI.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(network_input, network_output, epochs=25, batch_size=64, callbacks=callbacks_list)

Epoch 1/25
5395/5395 [==============================] - ETA: 425s - loss: 5.092 - ETA: 367s - loss: 5.067 - ETA: 354s - loss: 4.998 - ETA: 339s - loss: 4.964 - ETA: 328s - loss: 4.921 - ETA: 320s - loss: 4.891 - ETA: 314s - loss: 4.894 - ETA: 311s - loss: 4.913 - ETA: 307s - loss: 4.874 - ETA: 302s - loss: 4.847 - ETA: 298s - loss: 4.811 - ETA: 294s - loss: 4.793 - ETA: 290s - loss: 4.774 - ETA: 286s - loss: 4.765 - ETA: 282s - loss: 4.764 - ETA: 278s - loss: 4.749 - ETA: 275s - loss: 4.734 - ETA: 272s - loss: 4.713 - ETA: 269s - loss: 4.695 - ETA: 266s - loss: 4.671 - ETA: 262s - loss: 4.656 - ETA: 259s - loss: 4.645 - ETA: 255s - loss: 4.626 - ETA: 252s - loss: 4.610 - ETA: 248s - loss: 4.601 - ETA: 245s - loss: 4.579 - ETA: 241s - loss: 4.569 - ETA: 237s - loss: 4.556 - ETA: 234s - loss: 4.542 - ETA: 230s - loss: 4.532 - ETA: 226s - loss: 4.523 - ETA: 223s - loss: 4.517 - ETA: 218s - loss: 4.515 - ETA: 214s - loss: 4.515 - ETA: 211s - loss: 4.514 - ETA: 207s - loss: 4.511 - ETA: 203

5395/5395 [==============================] - ETA: 428s - loss: 4.364 - ETA: 425s - loss: 4.115 - ETA: 421s - loss: 4.044 - ETA: 415s - loss: 3.975 - ETA: 410s - loss: 3.888 - ETA: 406s - loss: 3.894 - ETA: 401s - loss: 3.882 - ETA: 396s - loss: 3.845 - ETA: 391s - loss: 3.881 - ETA: 385s - loss: 3.879 - ETA: 380s - loss: 3.880 - ETA: 375s - loss: 3.907 - ETA: 369s - loss: 3.892 - ETA: 363s - loss: 3.875 - ETA: 358s - loss: 3.864 - ETA: 353s - loss: 3.841 - ETA: 348s - loss: 3.833 - ETA: 343s - loss: 3.814 - ETA: 337s - loss: 3.817 - ETA: 333s - loss: 3.801 - ETA: 327s - loss: 3.800 - ETA: 322s - loss: 3.799 - ETA: 317s - loss: 3.805 - ETA: 312s - loss: 3.813 - ETA: 307s - loss: 3.822 - ETA: 302s - loss: 3.820 - ETA: 296s - loss: 3.819 - ETA: 291s - loss: 3.824 - ETA: 286s - loss: 3.823 - ETA: 280s - loss: 3.816 - ETA: 275s - loss: 3.821 - ETA: 270s - loss: 3.831 - ETA: 265s - loss: 3.839 - ETA: 260s - loss: 3.849 - ETA: 255s - loss: 3.857 - ETA: 249s - loss: 3.854 - ETA: 244s - loss: 3

5395/5395 [==============================] - ETA: 417s - loss: 4.369 - ETA: 412s - loss: 4.403 - ETA: 412s - loss: 4.313 - ETA: 410s - loss: 4.305 - ETA: 408s - loss: 4.347 - ETA: 404s - loss: 4.345 - ETA: 400s - loss: 4.335 - ETA: 395s - loss: 4.357 - ETA: 389s - loss: 4.425 - ETA: 384s - loss: 4.602 - ETA: 379s - loss: 4.619 - ETA: 374s - loss: 4.616 - ETA: 369s - loss: 4.596 - ETA: 364s - loss: 4.585 - ETA: 359s - loss: 4.588 - ETA: 354s - loss: 4.583 - ETA: 348s - loss: 4.566 - ETA: 342s - loss: 4.548 - ETA: 337s - loss: 4.538 - ETA: 332s - loss: 4.563 - ETA: 327s - loss: 4.641 - ETA: 322s - loss: 4.650 - ETA: 317s - loss: 4.663 - ETA: 311s - loss: 4.657 - ETA: 306s - loss: 4.654 - ETA: 301s - loss: 4.634 - ETA: 295s - loss: 4.621 - ETA: 290s - loss: 4.611 - ETA: 285s - loss: 4.607 - ETA: 280s - loss: 4.597 - ETA: 275s - loss: 4.587 - ETA: 270s - loss: 4.569 - ETA: 265s - loss: 4.558 - ETA: 260s - loss: 4.548 - ETA: 254s - loss: 4.542 - ETA: 249s - loss: 4.542 - ETA: 244s - loss: 4

5395/5395 [==============================] - ETA: 432s - loss: 4.453 - ETA: 429s - loss: 4.325 - ETA: 427s - loss: 4.338 - ETA: 420s - loss: 4.307 - ETA: 413s - loss: 4.286 - ETA: 408s - loss: 4.295 - ETA: 402s - loss: 4.303 - ETA: 397s - loss: 4.287 - ETA: 393s - loss: 4.257 - ETA: 387s - loss: 4.254 - ETA: 383s - loss: 4.265 - ETA: 377s - loss: 4.267 - ETA: 372s - loss: 4.279 - ETA: 366s - loss: 4.269 - ETA: 360s - loss: 4.267 - ETA: 355s - loss: 4.266 - ETA: 350s - loss: 4.269 - ETA: 344s - loss: 4.261 - ETA: 339s - loss: 4.260 - ETA: 334s - loss: 4.257 - ETA: 328s - loss: 4.256 - ETA: 323s - loss: 4.263 - ETA: 317s - loss: 4.268 - ETA: 312s - loss: 4.267 - ETA: 307s - loss: 4.258 - ETA: 302s - loss: 4.264 - ETA: 297s - loss: 4.262 - ETA: 292s - loss: 4.277 - ETA: 287s - loss: 4.275 - ETA: 281s - loss: 4.278 - ETA: 276s - loss: 4.273 - ETA: 271s - loss: 4.272 - ETA: 266s - loss: 4.274 - ETA: 260s - loss: 4.268 - ETA: 255s - loss: 4.265 - ETA: 250s - loss: 4.265 - ETA: 245s - loss: 4

5395/5395 [==============================] - ETA: 433s - loss: 4.049 - ETA: 429s - loss: 4.162 - ETA: 420s - loss: 4.142 - ETA: 413s - loss: 4.138 - ETA: 409s - loss: 4.157 - ETA: 405s - loss: 4.142 - ETA: 401s - loss: 4.151 - ETA: 397s - loss: 4.189 - ETA: 391s - loss: 4.193 - ETA: 386s - loss: 4.178 - ETA: 380s - loss: 4.209 - ETA: 375s - loss: 4.223 - ETA: 370s - loss: 4.214 - ETA: 365s - loss: 4.232 - ETA: 360s - loss: 4.225 - ETA: 355s - loss: 4.237 - ETA: 350s - loss: 4.242 - ETA: 344s - loss: 4.244 - ETA: 339s - loss: 4.241 - ETA: 333s - loss: 4.237 - ETA: 328s - loss: 4.229 - ETA: 323s - loss: 4.240 - ETA: 318s - loss: 4.251 - ETA: 313s - loss: 4.257 - ETA: 308s - loss: 4.256 - ETA: 302s - loss: 4.257 - ETA: 297s - loss: 4.255 - ETA: 291s - loss: 4.259 - ETA: 286s - loss: 4.259 - ETA: 281s - loss: 4.256 - ETA: 276s - loss: 4.260 - ETA: 271s - loss: 4.258 - ETA: 266s - loss: 4.252 - ETA: 260s - loss: 4.245 - ETA: 255s - loss: 4.250 - ETA: 250s - loss: 4.245 - ETA: 245s - loss: 4

In [ ]:
model = Sequential()
model.add(LSTM(512,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256)) 
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')


In [7]:
# Load the weights to each node
model.load_weights('weights-improvement-05-3.8923-bigger-2MIDI.hdf5')

In [11]:
start = np.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
prediction_output = []
pattern=list(pattern)
# generate 500 notes
for note_index in range(1500):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

In [12]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5

In [13]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='2midifileOutput.mid')

'2midifileOutput.mid'

In [5]:
s = converter.parse('.\Output\elsieOutput.mid')

In [6]:
s.show('text')

{0.0} <music21.stream.Part 0x16aa1684080>
    {0.0} <music21.note.Rest rest>
    {1.0} <music21.note.Note B->
    {1.5} <music21.note.Rest rest>
    {2.0} <music21.note.Note B->
    {2.5} <music21.note.Note B->
    {3.0} <music21.note.Note B->
    {3.5} <music21.note.Note B->
    {4.0} <music21.note.Note B->
    {4.5} <music21.note.Note B->
    {5.0} <music21.note.Note B->
    {5.5} <music21.note.Note B->
    {6.0} <music21.note.Note B->
    {6.5} <music21.note.Note B->
    {7.0} <music21.note.Note B->
    {7.5} <music21.note.Note B->
    {8.0} <music21.note.Note B->
    {8.5} <music21.note.Note B->
    {9.0} <music21.note.Note B->
    {9.5} <music21.note.Note B->
    {10.0} <music21.note.Note B->
    {10.5} <music21.note.Note B->
    {11.0} <music21.note.Note B->
    {11.5} <music21.note.Note B->
    {12.0} <music21.note.Note B->
    {12.5} <music21.note.Note B->
    {13.0} <music21.note.Note B->
    {13.5} <music21.note.Note B->
    {14.0} <music21.note.Note B->
    {14.5} <music21.n

    {250.5} <music21.note.Note B->


In [9]:
s = converter.parse('.\\Output\\2midifileOutput.mid')
s.show('text')

{0.0} <music21.stream.Part 0x16aa1a32438>
    {0.0} <music21.note.Rest rest>
    {1.0} <music21.chord.Chord C#4 E4 G#4>
    {1.5} <music21.note.Rest rest>
    {2.0} <music21.chord.Chord C#4 E4 G#4>
    {2.5} <music21.chord.Chord C#4 E4 G#4>
    {3.0} <music21.chord.Chord C#4 E4 G#4>
    {3.5} <music21.chord.Chord C#4 E4 G#4>
    {4.0} <music21.chord.Chord C#4 E4 G#4>
    {4.5} <music21.chord.Chord C#4 E4 G#4>
    {5.0} <music21.chord.Chord C#4 E4 G#4>
    {5.5} <music21.chord.Chord C#4 E4 G#4>
    {6.0} <music21.chord.Chord C#4 E4 G#4>
    {6.5} <music21.chord.Chord C#4 E4 G#4>
    {7.0} <music21.chord.Chord C#4 E4 G#4>
    {7.5} <music21.chord.Chord B4 C#4>
    {8.0} <music21.note.Note C#>
    {8.5} <music21.chord.Chord B4 C#4>
    {9.0} <music21.chord.Chord B4 C#4>
    {9.5} <music21.chord.Chord B4 C#4>
    {10.0} <music21.chord.Chord B4 C#4>
    {10.5} <music21.chord.Chord B4 C#4>
    {11.0} <music21.chord.Chord B4 C#4>
    {11.5} <music21.chord.Chord B4 C#4>
    {12.0} <music21.cho

    {250.5} <music21.chord.Chord B4 C#4>
